# PROJET AVION

## 1. Présentation Lappy / prédimensionnement d'avion

Lappy est un outil de prédimensionnement utilisé pour déterminer les caractérisitiques principales d'un avion d'affaire répondant à un cahier des charges donné. Ces caractéristiques sont déterminées en étudiants les points suivants :
- Aménagement, permettant d'accueillir passagers, bagages et équipages dans de bonnes conditions de confort,
- Première estimation de la masse de l'avion pour évaluer les surfaces de voilure nécessaires,
- Choix d'un moteur ayant la poussée nécessaire pour le décollage,
- Estimation du volume de carburant à embarquer pour assurer la mission,
- Positionnement de la voilure par rapport au centre de gravité pour assurer la stabilité et la manœuvrabilité de l'avion en vol 

Ce fichier doit être compléter en s'inspirant de l'exercice sur le ressort.

## 2. Initialisation
### a. Bibliothèque, variables globales et fonctions utiles
Nous commençons à lancer un sous-programme qui permet :
- de charger des bibliothèques 
- de définir des variables globales 
- de définir des fonctions :
    - affichage d'un tableau de valeurs
    - tracer graphique de l'avion


In [ ]:
%run LAPPY_Valeurs_et_fonctions_utiles.ipynb

In [ ]:
class Avion:
    "Définition d'un avion d'affaire"
class Trace:
    "définition d'un objet de type Trace"
    

### b. Initialisation, cahier des charges et valeurs initiales 

        *** valeur à modifier en fonction de votre cahier des charges ***

In [ ]:
def initialisation(avion):
    #Cahier des charges
    avion.mach_obj = 0.8        #Mach de croisière
    avion.zm_obj = 41000*0.3048 #Altitude de croisière [m]
    avion.npax_obj = 9          #Nombre de passager(s)
    avion.bfl_obj = 1200        #Longueur de piste au décollage objective [m]
    avion.dist_obj = 4800000    #Rayon d'action objectif [m]
    
    #Valeurs à initialiser
        # A compléter en fonction de votre processus de dimensionnement
    return
    
    

### c. Les différents modeleurs

Le programme est séparé en différent modeleur permettant de **affecter des valeurs d'entrée, calculer, analyser et corriger ** les caractéristiques de l'avion liées à :
- son aménagement, 
- sa géométrie, 
- sa propulsion, 
- son aérodynamisme,
- ses performances. 

In [ ]:
%run LAPPY_Modeleur_amenagement.ipynb
%run LAPPY_Modeleur_geometrie.ipynb
%run LAPPY_Modeleur_propulsion.ipynb
%run LAPPY_Modeleur_aerodynamique.ipynb
%run LAPPY_Modeleur_masse.ipynb
%run LAPPY_Modeleur_performances.ipynb

## 3. Calcul d'une définition complète

### a. Initialisation

Création d'une instance avion qu'on initialise avec les données du cahier des charges et des premières affectations de variables.

Création d'un objet "trace" permettant d'étudier l'évolution de certaines caractéristiques :

| Variable    | Libellé                                  | Unité     |
| ----------- | ---------------------------------------- | --------- |
|  qi         | Quantité carburant                       | kg        |
| srd         | Surface de référence dérive              | m2        |
| sremp       | Surface de référence empennage           | m2        |
| srv         | Surface référence voilure                | m2        |

La fonction **inc_trace** doit être appelée à chaque fois qu'on fait une correction sur l'avion et qu'on souhaite enregistrer cette correction dans la trace.

In [ ]:
avion=Avion()   # création d'une instance "avion" de la classe "Avion"
initialisation(avion)  # initialisation avec cahier des charges et valeurs initiales

trace=Trace()   # création d'une instance "trace" de la classe "Trace"
liste_att_trace=("qi","srd","sremp","srv","xrv")  # liste des variables à tracer

init_trace(trace)


### b. Aménagement
En fonction de valeurs recommandées, l'étude de l'aménagement conduit à définir quelques attributs de l'avion dont sa longueur de cabine.

In [ ]:
# Aménagement
affecte_entrees_amenagement(avion)
calcul_amenagement(avion)

print("Quelques caractéristiques de l'avion après son aménagement")
HTML(code_tableau_html_H_att([avion], ("npax_obj", "nbsalon", "lcab")))

### c. Géométrie
En fonction de valeurs recommandées, l'étude de la géométrie conduit à définir la voilure de l'avion.


In [ ]:
# Géométrie
affecte_entrees_geometrie(avion)
calcul_geometrie(avion)

### d. Propulsion
En fonction du choix du moteur, complète la définition de l'avion et permet une première représentation graphique.

In [ ]:
affecte_entrees_propulsion(avion)
calcul_propulsion(avion)
vue_dessus(avion)
vue_profil(avion)

### d. Aérodynamisme


In [ ]:
avion.mis_cz=0.34 # hypothèse car sera calculé grace au modeleur masse
affecte_entrees_aerodynamique(avion)
calcul_aerodynamique(avion)
analyse_finesse(avion)
calcul_foyer_aero(avion)

### e. Masse

In [ ]:
affecte_entrees_masse(avion)
calcul_masses(avion)

### f. Performance


In [ ]:
affecte_entrees_performances(avion)
print(analyse_performances_gv(avion))
print(analyse_performances_bv(avion))

## 4. Correction de la première définition

**A vous de définir des fonctions d'analyse et de correction, pour réaliser une première étape de correction **

N'oubliez pas d'appeler dans vos fonctions de correction inc_trace(avion,trace), afin que les évolutions des valeurs des attributs de l'avion soit enregistrées.

#### Caractéristiques de l'avion après les premieres corrections et évolution des paramètres

On peut ici afficher la définition actuelle des principaux paramètres de définition de l'avion et l'évolution de ces parametres suite aux corrections. 

In [ ]:
HTML(code_tableau_html_H_att([avion], ("srv", "srd", "sremp", "xrv","qi","mtow")))
courbe_iteration (trace, trace.srv, "Surf. ref voilure", "r")
courbe_iteration (trace, trace.qi, "quantité carburant", "b")
courbe_iteration (trace, trace.xrv, "position voilure", "g")
courbe_iteration (trace, trace.srd, "surface dérive", "y")
courbe_iteration (trace, trace.sremp, "surface empenage", "k")

## 5. Définition du processus de dimensionnement

Vous avez à définir un processus sous forme d'organigramme.
Vous pouvez utiliser une application en ligne pour créer votre organigramme. Par exemple https://mermaidjs.github.io/mermaid-live-editor/ qui à partir du code :


```
graph TD
A[Etape 1] -->B[Etape2]
B --> C{test}
C -->|OK| D[Etape3]
C -->|KO| E[correction]
E -->A
```

Vous réalise le graphe suivant :

![organigramme](Images/organigramme.JPG)

Vous avez ensuite à coder le processus en définissant des fonctions pour chaque étape suivant le modèle suivant.

In [ ]:
def etape1(avion, trace):
    # le code à executer
    # ....
    
    #Etape suivante
    etape2(avion, trace)

    
def etape2(avion, trace):
    # le code à executer
    # ....
    
    if <test KO>:
        # le code de la correction à executer
        # ....
        #Etape suivante
        etape1(avion, trace)
    else:
        #Etape suivante
        etape3(avion, trace)

# 6. Exécution du processus de dimensionnement

Il ne vous reste plus qu'à lancer le processus et d'afficher les résultats

In [ ]:
init_trace(trace)
etape1(avion, trace)
HTML(code_tableau_html_H_att([avion], ("srv", "srd", "sremp", "xrv","qi","mtow")))
courbe_iteration (trace, trace.srv, "Surf. ref voilure", "r")
courbe_iteration (trace, trace.qi, "quantité carburant", "b")
courbe_iteration (trace, trace.xrv, "position voilure", "g")
courbe_iteration (trace, trace.srd, "surface dérive", "y")
courbe_iteration (trace, trace.sremp, "surface empenage", "k")

In [ ]:
courbe_iteration (trace, trace.srv, "Surf. ref voilure", "r")
courbe_iteration (trace, trace.qi, "quantité carburant", "b")
courbe_iteration (trace, trace.xrv, "position voilure", "g")
courbe_iteration (trace, trace.srd, "surface dérive", "y")
courbe_iteration (trace, trace.sremp, "surface empenage", "k")